In [1]:
!ln -s /root/DF-VO/dataset/custom

In [21]:
import os

In [22]:
os.environ['DISPLAY']

'X'

In [19]:
!echo 'y\n0' | python run.py -c ./options/custom_configuration.yml

Save result in results/0? [y/n]==> initialize LiteFlowNet with [model/network-default.pytorch]: 
==> Start VO
  0%|                                                    | 0/10 [00:00<?, ?it/s]: cannot connect to X server X
